In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import chardet


In [2]:
# Laden Sie Ihre Daten
df = pd.read_csv('../data/raw/gpd_v2_20220427.csv', delimiter=',', encoding='utf-8')

In [4]:

import os
# Teilen Sie die Daten in populistisch und nicht populistisch
print(df.columns)

populist_files = df[df['rubricgrade'] == 2]['merging_variable']
non_populist_files = df[df['rubricgrade'] == 0]['merging_variable']

# Verzeichnis mit Ihren Textdateien
text_files_directory = '/Users/furkansaygin/Documents/Studium/ML Projekt 2/Adv.ML-Project/data/raw/speeches_20220427/'

# Funktion zum Lesen der Dateiinhalte
def read_file_contents(filename):
    filepath = os.path.join(text_files_directory, filename)
    
    if os.path.isfile(filepath):
        with open(filepath, 'rb') as file:
            rawdata = file.read()
        encoding = chardet.detect(rawdata)['encoding']
        with open(filepath, 'r', encoding=encoding) as file:
            return file.read().replace('\n', '')

    else:
        return None  # oder ein leerer String '', je nachdem, was in Ihrem Fall am besten passt


# Entferne alle Zeilen mit fehlenden Werten in 'merging_variable'
df = df.dropna(subset=['merging_variable'])

# Hinzufügen der Dateiinhalte zu Ihrem DataFrame
df['file_contents'] = df['merging_variable'].apply(read_file_contents)


# Speichere den bereinigten Datensatz
df.to_csv('../data/processed/gpd_processed.csv', index=False)





Index(['merging_variable', 'country', 'leader', 'party', 'lr', 'president',
       'term', 'startofterm', 'yearbegin', 'endofterm', 'yearend',
       'speechtype', 'speechnum', 'codernum', 'rubricgrade', 'averagerubric',
       'totalaverage', 'wb_region', 'region'],
      dtype='object')
